In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

# Import libararies for metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report 


# import StandardSclaer and OneHotEncoder
from sklearn.preprocessing import StandardScaler,OneHotEncoder

# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Import LogisticRegression library
from sklearn.linear_model import LogisticRegression

# Import BalancedRandomForestClassifier from imblearn
from imblearn.ensemble import BalancedRandomForestClassifier

# Import KNeighborsClasifier library
from sklearn.neighbors import KNeighborsClassifier

# Import SVC library
from sklearn.svm import SVC

# Import Adaboost library
from sklearn.ensemble import AdaBoostClassifier

#import libraries to save the model
import pickle
import joblib


import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Credit Risk information to Google Colab
url = 'https://raw.githubusercontent.com/DIsaacman/Dream-Team/dataset/Resources/credit_risk_dataset.csv'

# Read CSV
df = pd.read_csv(url)

# Review the DataFrame
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [3]:
# drop the columns we don't want to use in our models
consolidated_df = df.drop(columns=[
       'person_emp_length', 'loan_intent', 'loan_grade',
       'loan_int_rate', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'])

In [4]:
# Create a list of categorical variables 
categorical_variables = list(consolidated_df.dtypes[consolidated_df.dtypes=="object"].index)

# Display the categorical variables list
categorical_variables

['person_home_ownership']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [6]:
# fit and transform the categorial variable
encoded_data = enc.fit_transform(consolidated_df[categorical_variables])

In [7]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
columns=enc.get_feature_names_out(categorical_variables)
)

# Review the DataFrame
encoded_df.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT
0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


In [8]:
# create a new df that does not include those columns we converted with one_hot_encoder (all numerical variables from original dataset)

numerical_variables_df = consolidated_df.drop(columns=categorical_variables)

#review the dataframe
numerical_variables_df.head()

,person_age,person_income,loan_amnt,loan_status
0,22,59000,35000,1
1,21,9600,1000,0
2,25,9600,5500,1
3,23,65500,35000,1
4,24,54400,35000,1


In [9]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame

encoded_df = pd.concat([encoded_df, numerical_variables_df], axis=1)

# Review the Dataframe
encoded_df.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,person_age,person_income,loan_amnt,loan_status
0,0.0,0.0,0.0,1.0,22,59000,35000,1
1,0.0,0.0,1.0,0.0,21,9600,1000,0
2,1.0,0.0,0.0,0.0,25,9600,5500,1
3,0.0,0.0,0.0,1.0,23,65500,35000,1
4,0.0,0.0,0.0,1.0,24,54400,35000,1


In [10]:
# separate the data into features and target

y = encoded_df["loan_status"]

In [11]:
y[:7]

0    1
1    0
2    1
3    1
4    1
5    1
6    1
Name: loan_status, dtype: int64

In [12]:
X = encoded_df.drop(columns="loan_status")

In [13]:
X.head()

,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,person_age,person_income,loan_amnt
0,0.0,0.0,0.0,1.0,22,59000,35000
1,0.0,0.0,1.0,0.0,21,9600,1000
2,1.0,0.0,0.0,0.0,25,9600,5500
3,0.0,0.0,0.0,1.0,23,65500,35000
4,0.0,0.0,0.0,1.0,24,54400,35000


In [14]:
# check the value_counts to see whether there is an oversampling issue
y.value_counts()

0    25473
1     7108
Name: loan_status, dtype: int64

In [15]:
# Split the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Instantiate the random oversampler model
random_oversampler = RandomOverSampler(random_state=1)
# Fit the training data to the random_oversampler model
X_resampled, y_resampled = random_oversampler.fit_resample(X_train, y_train)

In [18]:
y_resampled.value_counts()

0    19113
1    19113
Name: loan_status, dtype: int64

---------

### Logistic Regression Model

In [19]:
# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(random_state=1)
# Fit the model using the resampled training data
lr_model = logistic_regression_model.fit(X_resampled, y_resampled)
# Make a prediction using the testing data
predictions_oversampled = logistic_regression_model.predict(X_test)

In [20]:
balanced_accuracy_score(y_test, predictions_oversampled)

0.6767254220456802

In [21]:
confusion_matrix(y_test, predictions_oversampled)

array([[4089, 2271],
       [ 517, 1269]])

In [22]:
print(classification_report_imbalanced(y_test, predictions_oversampled))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.64      0.71      0.75      0.68      0.45      6360
          1       0.36      0.71      0.64      0.48      0.68      0.46      1786

avg / total       0.77      0.66      0.70      0.69      0.68      0.46      8146



------------

### Balanced Random Forest Model

In [23]:
# Instantiate a BalancedRandomForestClassifier instance
brf = BalancedRandomForestClassifier()

# Fit the model to the training data
brf.fit(X_resampled, y_resampled)

BalancedRandomForestClassifier()

In [24]:
BalancedRandomForestClassifier()

BalancedRandomForestClassifier()

In [25]:
# Predict labels for testing features
y_pred = brf.predict(X_test_scaled)

In [26]:
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      6360
           1       0.00      0.00      0.00      1786

    accuracy                           0.78      8146
   macro avg       0.39      0.50      0.44      8146
weighted avg       0.61      0.78      0.68      8146



In [27]:
balanced_accuracy_score(y_test, y_pred)

0.5

----------

### KNN Model

In [28]:
knn_model = KNeighborsClassifier(n_neighbors=3) 

In [29]:
knn_model.fit(X_resampled, y_resampled) 

KNeighborsClassifier(n_neighbors=3)

In [30]:
 # Create predictions
y_pred_knn = knn_model.predict(X_test_scaled)

# Review the predictions
y_pred_knn

array([1, 1, 1, ..., 1, 1, 1])

In [31]:
print(classification_report(y_pred_knn,y_test)) 

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.22      0.36      8146

    accuracy                           0.22      8146
   macro avg       0.50      0.11      0.18      8146
weighted avg       1.00      0.22      0.36      8146



In [32]:
balanced_accuracy_score(y_pred_knn,y_test)

0.21924871102381538

------------

### Adaboost Model

In [33]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=100,
                         learning_rate=1)

In [34]:
# Train Adaboost Classifer
abc_model = abc.fit(X_resampled, y_resampled)

In [35]:
#Predict the response for test dataset
abc_pred = abc_model.predict(X_test_scaled)

In [36]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, abc_pred))

Accuracy: 0.2871347900810214


In [37]:
print(classification_report(y_test, abc_pred)) 


              precision    recall  f1-score   support

           0       0.93      0.09      0.17      6360
           1       0.23      0.97      0.37      1786

    accuracy                           0.29      8146
   macro avg       0.58      0.53      0.27      8146
weighted avg       0.77      0.29      0.22      8146



In [38]:
balanced_accuracy_score(abc_pred,y_test)

0.579627309401171

----------

### Saving The Chosen Model

In [39]:
saved_model = pickle.dumps(abc_model)

In [40]:
# save the model to disk
filename = 'finalized_pickle_model.sav'
pickle.dump(abc_model, open(filename, 'wb'))

### Save the model using joblib

In [41]:
# save the model to disk

filename = 'finalized_joblib_model.sav'
joblib.dump(abc_model, filename)

['finalized_joblib_model.sav']